In [2]:
%load_ext autoreload
%autoreload 2
import pandas as pd
import numpy as np
import pandas_helper
pd.set_option('display.max_rows', 200)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
df = pandas_helper.load_data("data/wikilanguage.tsv")

In [5]:
len(df)

26838727

In [8]:
pandas_helper.resolve_label(df, "Canada")

Index(['Q5029279', 'Q17672891', 'Q21287484', 'Q28457929', 'Q2608363',
       'Q13265795', 'Q21188718', 'Q2684010', 'Q5029265', 'Q2569593', 'Q16',
       'Q5029291', 'Q5029285', 'Q28977393', 'Q257304', 'Q1031917', 'Q5029278',
       'Q5029289', 'Q13265725', 'Q14624136'],
      dtype='object', name='concept_id')

In [ ]:
dataset[dataset.loc[dataset['sample_label'] == 'George Washington']]

In [ ]:
# Memory Hungry Function
top_concepts = pandas_helper.best_concepts(
    dataset, sample=0.1, n=500
)

In [ ]:
films = instance_of(dataset, Concepts.FILM).drop(
    index=[
        "Q2537294", # All-Russia State Television and Radio Broadcas
        "Q11659568", # Minami no Teiō 
    ]
)

In [ ]:
games = instance_of(dataset, Concepts.VIDEO_GAME).drop(
    index=[
        
    ]
)

In [ ]:
top_ranked(games, "frwiki")

In [ ]:
marginals = top_kl(films, "enwiki", "jawiki", marginals=True, importance_weight=1)["enwiki_title"].reset_index()
no_marginals = top_kl(films, "enwiki", "jawiki", marginals=True, importance_weight=2)["enwiki_title"].reset_index()
pandas.concat((marginals, no_marginals), axis=1)